<center><h1><strong>WeRateDogs Data Wrangling</strong></h1></center>

# Introduction

This project started as an assessment for the **Udacity Data Analysis Professional Track**.

The dataset is the tweet archive of Twitter user [@dog_rates](https://twitter.com/dog_rates), also known as 
[WeRateDogs](https://en.wikipedia.org/wiki/WeRateDogs). WeRateDogs is a Twitter account that rates people's dogs with a humorous comment about the dog. These ratings almost always have a denominator of 10. The numerators, though? Almost always greater than 10. 11/10, 12/10, 13/10, etc. Why? Because ["they're good dogs Brent."](https://knowyourmeme.com/memes/theyre-good-dogs-brent) WeRateDogs has over 4 million followers and has received international media coverage.

<center><img width="50%" src ="dog-rates-social.jpg"/></center>

The data wrangling process was achieved using python and the following packages (Libraries):

- [NumPy](https://numpy.org/)
- [Panadas](https://pandas.pydata.org/)
- [Requests](https://requests.readthedocs.io/en/master/)
- [Tweepy](https://www.tweepy.org/)
- [json](https://docs.python.org/3/library/json.html)


# 1. Gathering

Three sources of data were gathered for this Project:
1. Twitter acrhive provided by WeRateDogs
2. Image predictions from the neural network data downloaded programatically.
3. Information from the Twitter API scraped programatically.

We begin by importing the required libraries and packages that will help us in our data wranglling and analysis

In [1]:
#import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.dates as mdates
import seaborn as sns
import os
import requests
import tweepy
import json
import re

## 1.1 The WeRateDogs Twitter Archive

- The 'twitter-archive-enhanced.csv' file was provided by Udacity and downloaded manually.
- Then it was easily read into a pandas DataFrame named `df_archive`.

In [2]:
df_archive = pd.read_csv('twitter-archive-enhanced.csv')
df_archive.head(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None


## 1.2 The Tweet image predictions

- The 'image-predictions.tsv' shows the results of a neural network that detects what breed of dog (or other object, animal, etc.) is present in each tweet.   
- The file was downloaded programmatically using the Requests python library and then saved to a local file.
- The file name was extracted from the url using the split function.
- The file was then read into a pandas DataFrame `df_pred` while specifying that it's a tsv by setting the `sep` parameter to `'\t'`.

In [3]:
# Download image predictions file if it doesn't exist
pred_url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
file_name = pred_url.split('/')[-1]

# only download the file if the file doesn't already exist on disk.
if os.path.exists(file_name):
    print(f"File {file_name} already exists")
else:
    r = requests.get(pred_url)
    # write the reponse to a local file
    with open('image-predictions.tsv','wb') as file:
        file.write(r.content)
        print(f"successfully downloaded {file_name} and written to disc")

File image-predictions.tsv already exists


In [4]:
# Read the image predictions file into a DataFrame
df_pred = pd.read_csv('image-predictions.tsv',sep='\t')
df_pred.sample(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1907,852553447878664193,https://pbs.twimg.com/media/C9Tg1bPW0AkAMDI.jpg,1,bloodhound,0.186498,True,Brabancon_griffon,0.139028,True,Rottweiler,0.125940,True
1933,859607811541651456,https://pbs.twimg.com/media/C-3wvtxXcAUTuBE.jpg,1,golden_retriever,0.895529,True,Irish_setter,0.024099,True,Labrador_retriever,0.019285,True
1544,791672322847637504,https://pbs.twimg.com/media/CvyVxQRWEAAdSZS.jpg,1,golden_retriever,0.705092,True,Labrador_retriever,0.219721,True,kuvasz,0.015965,True


## 1.3 Twitter API 

- A Twitter Developer account was created and used to qurey more data using the tweet ids from the `df_archive` DataFrame.
- The Twitter API was used to query each tweet's JSON data using Python's Tweepy library and stored each tweet's entire set of JSON data as a line in a file called tweet_json.txt file.
- The 'tweet._json.txt' file line by line into a pandas DataFrame `df_api` with the following columns:
    - tweet ID.
    - retweet count. 
    - favorite count. 

- Note: Twitter API keys, secrets, and tokens were read from an external file and were not submitted with the project.

In [5]:
tweet_ids = df_archive['tweet_id']
json_file = 'tweet_json.txt'


# only Query the API if the tweet_json.txt file doesn't exist
if os.path.exists(json_file):
    print(f"File {json_file} already exists")
else:
    # import the keys files where the API keys are stored and it is not submitted with the project
    import keys
    # Twitter API authorization 
    auth = tweepy.OAuthHandler(keys.consumer_key, keys.consumer_secret)
    auth.set_access_token(keys.access_token, keys.access_secret)

    api = tweepy.API(auth)
    
    count = 0
    fails_dict = {}
    # Save each tweet's returned JSON as a new line in a .txt file
    with open(json_file, 'w') as outfile:
        # Loop through each tweet and write its information into a separate line of the .txt file
        for tweet_id in tweet_ids:
            count += 1
            print(str(count) + ": " + str(tweet_id))
            try:
                tweet = api.get_status(tweet_id, tweet_mode='extended')
                print("Success")
                json.dump(tweet._json, outfile)
                outfile.write('\n')
            except tweepy.TweepError as e:
                print("Fail")
                fails_dict[tweet_id] = e
                pass
    print(f"File {json_file} successfully written to Disk")
    print("Some tweets returned the following errors")
    print(fails_dict)



File tweet_json.txt already exists


In [6]:
# read .txt file line by line into a pandas DataFrame

labels = ['tweet_id','retweet_count','favorite_count']

df_api = pd.DataFrame(columns=labels)

with open(json_file) as file:
    for line in file:
        data = json.loads(line)
        df_api = df_api.append({
            'tweet_id':data['id'],
            'retweet_count':data['retweet_count'],
            'favorite_count':data['favorite_count']
            },ignore_index=True)
        
df_api.sample(5)

,tweet_id,retweet_count,favorite_count
1186,672248013293752320,559,1626
117,869227993411051520,3397,18958
420,820314633777061888,554,3312
249,842846295480000512,3498,14887
1223,671518598289059840,272,887


## 1.4 Gathering Output
 
The three files were successfully downloded, written to a local file and read into the following pandas DataFrames:
- `df_archive`:  Data from the WeRateDogs Twitter archive.
- `df_pred`:  Data from the image predictions from the neural network.
- `df_api`:  Data scraped from the Twitter API.


# 2. Assessing the Data

We start assesing the data to try to find any Quality or tidiness issues. 

The aim is to asses all three dataframes with respect to Quality dimensions:
 - Completeness
 - Validity
 - Accuracy
 - Consistency
 
and the following tidiness rules:
- Each variable forms a column.
- Each observation forms a row.
- Each type of observational unit forms a table.

## 2.1 Twitter Archive

using the `info` method we try to find missing data and look to see if we have non-original tweets (retweets and replies). Also check the is we already have the correct data types for all columns.

In [7]:
df_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [8]:
df_archive.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
270,841314665196081154,NaN,NaN,2017-03-13 15:47:01 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",This is Max. There's no way in h*ck you're tak...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/841314665...,13,10,Max,None,None,None,None
1266,709852847387627521,NaN,NaN,2016-03-15 21:24:41 +0000,"<a href=""http://twitter.com/download/iphone"" r...",*lets out a tiny whimper and then collapses* ....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/709852847...,12,10,None,None,None,None,None
2209,668623201287675904,NaN,NaN,2015-11-23 02:52:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Jomathan. He is not thrilled about the...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/668623201...,10,10,Jomathan,None,None,None,None
2218,668507509523615744,NaN,NaN,2015-11-22 19:13:05 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a Birmingham Quagmire named Chuk. Love...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/668507509...,10,10,a,None,None,None,None
503,813066809284972545,NaN,NaN,2016-12-25 17:00:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tyr. He is disgusted by holiday traffi...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/813066809...,12,10,Tyr,None,None,None,None


In [9]:
# check for duplicated entries
sum(df_archive['tweet_id'].duplicated())

0

It si clear that we have 181 tweets that are retweets and 78 tweets that are replies. That means these are not original tweets and will need to be filtered out.

In [10]:
df_archive['rating_denominator'].value_counts().keys()

Int64Index([10, 11, 50, 80, 20, 2, 16, 40, 70, 15, 90, 110, 120, 130, 150, 170,
            7, 0],
           dtype='int64')

In [11]:
df_archive[df_archive['rating_denominator'] != 10].sample(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1351,704054845121142784,NaN,NaN,2016-02-28 21:25:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a whole flock of puppers. 60/50 I'll ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/704054845...,60,50,a,None,None,None,None
313,835246439529840640,8.352460e+17,26259576.0,2017-02-24 21:54:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@jonnysun @Lin_Manuel ok jomny I know you're e...,NaN,NaN,NaN,NaN,960,0,None,None,None,None,None
784,775096608509886464,NaN,NaN,2016-09-11 22:20:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: After so many requests, this is...",7.403732e+17,4.196984e+09,2016-06-08 02:41:38 +0000,https://twitter.com/dog_rates/status/740373189...,9,11,None,None,None,None,None


In [12]:
df_archive['rating_numerator'].value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

The rating denominator was not always equal to 10 and also the rating numerator had some inconsistent values either too small or too large. After examining the data set by filtering the illogical and exploring both their text and their images, it was deduced that the reasons for the issues in these ratings were:
- The ratings of pack of dogs is added together making the denominator multiples of 10.
- The extraction process assumed that '/' character only appears once in the text and that it always indicates rating which was found not to be true.


In [13]:
df_archive['name'].value_counts()

None       745
a           55
Charlie     12
Oliver      11
Cooper      11
          ... 
Rinna        1
Jazz         1
Lugan        1
Evy          1
Sobe         1
Name: name, Length: 957, dtype: int64

In [14]:
names = df_archive['name'].value_counts().keys()
not_names = []

for name in names:
    if name.islower() or len(name) < 2:
        not_names.append(name)
print(not_names)

['a', 'the', 'an', 'very', 'quite', 'one', 'just', 'mad', 'not', 'getting', 'actually', 'my', 'incredibly', 'by', 'O', 'life', 'unacceptable', 'officially', 'his', 'this', 'all', 'infuriating', 'old', 'light', 'space', 'such']


The name column has a lot of missing values that are indicated as "None" in a string format.
Also, it contained a lot of words that should not be considered names. that probably happened because the extraction process assumed that names only and must come after the words "this is" which is not always true. 

In [15]:
df_archive[['doggo','floofer','pupper','puppo']].value_counts()

doggo  floofer  pupper  puppo
None   None     None    None     1976
                pupper  None      245
doggo  None     None    None       83
None   None     None    puppo      29
doggo  None     pupper  None       12
None   floofer  None    None        9
doggo  floofer  None    None        1
       None     None    puppo       1
dtype: int64

- The dog stages columns ('doggo', 'flooofer', 'pupper', 'puppo') are values and not variables. They all represent different dog stages. They should be one column instead.
- Some entries has more than one dog stage:
    - 12 entry has both doggo and pupper.
    - 1 entry has both doggo and floofer.
- Missing values are written as "None" in a string format instead on nulls.

In [16]:
df_archive['source'].value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

The 'source' column contains the whole HTML 'a' tag while the source is only the inner text of the tag.

## 2.2 Tweet Image Predictions

In [17]:
df_pred.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
901,700002074055016451,https://pbs.twimg.com/media/CbboKP4WIAAw8xq.jpg,1,Chihuahua,0.369488,True,schipperke,0.243367,True,pug,0.161614,True
2064,890006608113172480,https://pbs.twimg.com/media/DFnwSY4WAAAMliS.jpg,1,Samoyed,0.957979,True,Pomeranian,0.013884,True,chow,0.008167,True
1195,740365076218183684,https://pbs.twimg.com/media/CkZOGhJWsAAHvPv.jpg,1,bow_tie,0.246313,False,Windsor_tie,0.172446,False,mushroom,0.137516,False
1701,817120970343411712,https://pbs.twimg.com/media/C1b_LSYUsAAJ494.jpg,1,Saluki,0.568809,True,Afghan_hound,0.229352,True,golden_retriever,0.157130,True
1120,726828223124897792,https://pbs.twimg.com/media/ChY2aHyWMAAbNQE.jpg,1,miniature_pinscher,0.255327,True,Border_terrier,0.181279,True,Labrador_retriever,0.125185,True


In [18]:
sum(df_pred['tweet_id'].duplicated())

0

In [19]:
df_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [20]:
df_pred['img_num'].value_counts()

1    1780
2     198
3      66
4      31
Name: img_num, dtype: int64

- prediction levels are seperated in different columns while they actually all represent one variable "prediction_level" 
- Even though some tweets have more than one image but no duplicates were found.

## 2.3 Twitter API

In [21]:
df_api

,tweet_id,retweet_count,favorite_count
0,892420643555336193,7461,35352
1,892177421306343426,5537,30593
2,891815181378084864,3664,23025
3,891689557279858688,7637,38647
4,891327558926688256,8228,36916
...,...,...,...
1527,666049248165822465,39,95
1528,666044226329800704,124,264
1529,666033412701032449,39,108
1530,666029285002620928,41,118


In [22]:
df_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1532 entries, 0 to 1531
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tweet_id        1532 non-null   object
 1   retweet_count   1532 non-null   object
 2   favorite_count  1532 non-null   object
dtypes: object(3)
memory usage: 36.0+ KB


- The 'retweet_count' and 'favorite_count' columns have the wrong datatype. 
- The index column here is the same as the `df_archive` dataframe and they should be merged together.

## 2.4 Assesing Output

After the data was assessed both visually and programmatically, The following issues were concluded:

### Quality

#### Completeness: 
do we have all of the records that we should? Do we have missing records or not? Are there specific rows, columns, or cells missing?

`df_archive`
- 'expanded_urls' have missing values. taht means they don't have image information.

`df_api`
- The API query failed to retrieve data for 25 tweets because they are either no longer available or access is denied.


#### Validity: 
we have the records, but they're not valid, i.e., they don't conform to a defined schema. A schema is a defined set of rules for data. These rules can be real-world constraints (e.g. negative height is impossible) and table-specific constraints (e.g. unique key constraints in tables).

`df_archive`

- The 'is_reply_status_id' and 'retweeted_status_id' columns have non null values showing that 181 entries are retweets and 78 entries are replies. i.e; not original retweets.
- Also 'expanded_urls' have missing values which means they don't contain images.

#### Accuracy: 
inaccurate data is wrong data that is valid. It adheres to the defined schema, but it is still incorrect. Example: a patient's weight that is 5 lbs too heavy because the scale was faulty.

`df_archive`

- the 'name' column has some values that are not actual name. such as:  
'a', 'the', 'an', 'very', 'one', 'quite', 'just', 'mad', 'actually', 'getting', 'not', 'infuriating', 'space', 'unacceptable', 'his', 'old', 'O', 'life', 'this', 'incredibly', 'such', 'light', 'officially', 'all', 'by', 'my'.
- 12 entries have both lables 'doggo' and 'pupper'
- 1 entry have both labels 'doggo' and 'floofer

#### Consistency: 
inconsistent data is both valid and accurate, but there are multiple correct ways of referring to the same thing. Consistency, i.e., a standard format, in columns that represent the same data across tables and/or within tables is desired.

`df_archive`
- not all 'rating_denominator' values are 10.
- The 'rating_numerator' column has some very low or very large values.
- 'timestamp' and 'retweeted_status_timestamp' column is has type of srting. should be datetime.
- the 'source' column values are the complete 'a' tag. Should be reduced to only be actual source values:  
    iPhone, Vine, Web Client and TweetDeck
- The 'name' columns missing values are labeled 'None'.

`df_api`
- the 'tweet_id', 'retweet_count' and 'favorite_count' are of type string. Should be integers.


### Tidiness

`df_archive`
- The dog stages columns ('doggo', 'flooofer', 'pupper', 'puppo') should be one column instead.

`df_pred`
- Multiple predictions exist in the same row. the doean't follow the tisiness rule 'every row is an observation'. Each prediction should be its own row instead.

`df_api`
- The observation row here is the tweet defined by the 'tweet_id' that is the same as in `df_archive`. The two tables should be one table instead. 

# 3. Cleaning

## 3.1 Twitter Archive

In [23]:
#Copying the original DataFrame into a new one to be cleaned
df_archive_clean = df_archive.copy()

#### Define
* Convert the 'timestamp' and 'retweeted_status_timestamp' columns from string to datetime using the `pd.to_datetime` function.

#### Code

In [24]:
df_archive_clean['timestamp'] = pd.to_datetime(df_archive_clean['timestamp'])
df_archive_clean['retweeted_status_timestamp'] = pd.to_datetime(df_archive_clean['retweeted_status_timestamp'])

#### Test

In [25]:
# check that the 'timestamp' and 'retweeted_status_timestamp' columns are changed to datetime
df_archive_clean[['timestamp','retweeted_status_timestamp']].dtypes

timestamp                     datetime64[ns, UTC]
retweeted_status_timestamp    datetime64[ns, UTC]
dtype: object

#### Define

* Remove all tweets that are not orignal tweets, i.e; tweets that are retweets or replies. So, the `df_archive_clean` is filtered to only the rows where the columns 'retweeted_status_id' and 'in_reply_to_status_id' are null values.

#### Code

In [26]:
df_archive_clean = df_archive_clean[df_archive_clean['retweeted_status_id'].isnull()]
df_archive_clean = df_archive_clean[df_archive_clean['in_reply_to_status_id'].isnull()]

#### Test

In [27]:
df_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    2097 non-null   int64              
 1   in_reply_to_status_id       0 non-null      float64            
 2   in_reply_to_user_id         0 non-null      float64            
 3   timestamp                   2097 non-null   datetime64[ns, UTC]
 4   source                      2097 non-null   object             
 5   text                        2097 non-null   object             
 6   retweeted_status_id         0 non-null      float64            
 7   retweeted_status_user_id    0 non-null      float64            
 8   retweeted_status_timestamp  0 non-null      datetime64[ns, UTC]
 9   expanded_urls               2094 non-null   object             
 10  rating_numerator            2097 non-null   int64           

#### Define

* Remove the rows with empty 'expanded_urls'. These are the tweets that don't have image information.

#### Code

In [28]:
df_archive_clean = df_archive_clean[df_archive_clean['expanded_urls'].notna()]

#### Test

In [29]:
sum(df_archive_clean['expanded_urls'].isnull())

0

#### Define

- Remove the columns 'in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp' as they are no longer relevant

#### Code

In [30]:
cols = ['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp',]

df_archive_clean.drop(cols, axis=1, inplace=True)

#### Test

In [31]:
df_archive_clean.columns

Index(['tweet_id', 'timestamp', 'source', 'text', 'expanded_urls',
       'rating_numerator', 'rating_denominator', 'name', 'doggo', 'floofer',
       'pupper', 'puppo'],
      dtype='object')

#### Define

* Combine the dog stages columns ('doggo', 'flooofer', 'pupper', 'puppo') should be one column 'dog_stage' instead. This was done by appliyng a custom funcion `dog_stage` to the whole DataFrame that returns the interpreted dog_stage and convert the 'None' string to a null value
* for cases where 'doggo' and other value exist. 'doggo' will be chosen as 'doggo' is the older dog and dogs can only grow up. So, 'doggo' will definetly be the more valid value.

#### Code

In [32]:
def dog_stage(row):
    """
    Returns the dog stage based on the values of other columns
    """
    stages = ['doggo','floofer','pupper','puppo']
    for stage in stages:
        if row[stage].lower().strip() == stage:
            return stage
    return np.NaN

df_archive_clean['dog_stage'] = df_archive_clean.apply(lambda x: dog_stage(x), axis=1)

#### Test

In [33]:
df_archive_clean['dog_stage'].value_counts(dropna=False)

NaN        1759
pupper      220
doggo        83
puppo        23
floofer       9
Name: dog_stage, dtype: int64

#### Define

* Remove the  columns ('doggo', 'floofer', 'pupper', 'puppo') as they are no longer relevant and now combined into one column `dog_stage`

#### Code

In [34]:
cols = ['doggo', 'floofer', 'pupper', 'puppo']

df_archive_clean.drop(cols, axis=1, inplace=True)

#### Test

In [35]:
df_archive_clean.columns

Index(['tweet_id', 'timestamp', 'source', 'text', 'expanded_urls',
       'rating_numerator', 'rating_denominator', 'name', 'dog_stage'],
      dtype='object')

#### Define

- Fixing the 'rating_numerator' and 'rating_denominator' columns by addressing the possible error causes:
    - for the pack of dogs, the raing denominator is usually a multiple of 10. if that's the case, a factor is calculated to return the rating back to the base of 10.
    - for the errors in extractions of data when the text sontains more than one '/' character. The text is searched using regex for a valid rating with a denominator of 10 and then the rating numerator and denominator are extracted as a result.
    - if there's only one '/' character in the text, a base rating is assumed of 10/10.
    - for the decimal point ratings, the integer value before the decimal point is extracted and considered as the valid rating numerator.
    - only two teets were left with exceptional high scores. They were not changes as they were valid ratings. One for a real dog and another for the famous artist "Snoop Dog". 

#### Code

In [36]:
def fix_rating(row):
    if row['rating_denominator'] != 10:
        if row['rating_denominator'] % 10 ==0:
            num = row['rating_denominator']/10
            row['rating_numerator']=int(row['rating_numerator']/num)
            row['rating_denominator'] = 10
        else:
            prog = re.compile('../..')
            results = re.findall(prog, row['text'])
            for result in results:
                if result[-2:] == '10':
                    row['rating_numerator']= int(result[:2])
                    row['rating_denominator'] = 10
                    results.remove(result)
            if len(results):
                row['rating_numerator']= 10
                row['rating_denominator'] = 10
                
    else:
        prog = re.compile('\d*\.\d*/10')
        results = re.findall(prog, row['text'])
        for result in results:
            if result[-2:] == '10':
                try:
                    row['rating_numerator'] = int(result.split('.')[0].strip())
                except:
                    row['rating_numerator'] = 10
        
    
    return row

    
df_archive_clean = df_archive_clean.apply(lambda x: fix_rating(x), axis=1)

df_archive_clean[['tweet_id','text','rating_numerator','rating_denominator']].query('rating_numerator > 20 or  rating_denominator !=10')

,tweet_id,text,rating_numerator,rating_denominator
979,749981277374128128,This is Atticus. He's quite simply America af....,1776,10
2074,670842764863651840,After so many requests... here you go.\n\nGood...,420,10


#### Test

In [37]:
df_archive_clean['rating_denominator'].value_counts()

10    2094
Name: rating_denominator, dtype: int64

In [38]:
df_archive_clean['rating_numerator'].value_counts()

12      487
10      451
11      416
13      287
9       151
8        97
7        51
14       38
5        33
6        32
3        19
4        15
2        10
1         4
420       1
1776      1
0         1
Name: rating_numerator, dtype: int64

#### Define

- The 'name' column is fixed by applying a custom function `fix_name` that tries to find other possibe names in the tweet's text. If no possible name was found in the text the name value is set to null.

#### Code

In [39]:
not_names = ['a', 'the', 'an', 'very', 'one', 'quite', 'just', 'mad', 'actually', 'getting', 'not', 
                 'infuriating', 'space', 'unacceptable', 'his', 'old', 'O', 'life', 'this', 'incredibly', 
                 'such', 'light', 'officially', 'all', 'by', 'my']  

def fix_name(row):
    """
    Returns the row withthe corrected name if found and Null if no name was found
    """
    if row['name'] in not_names or row['name'] == 'None':
        try:
            pattern = re.compile(r'(?:name(?:d)?)\s{1}(?:is\s)?([A-Za-z]+)')
            new_name = re.findall(pattern, row['text'])[0]
            row['name'] = new_name 

        except IndexError:
            row['name'] = np.NaN
    return row


df_archive_clean = df_archive_clean.apply(lambda x: fix_name(x), axis=1)

#### Test

In [40]:
df_archive_clean[['tweet_id','text','name']].query('name in @not_names')

,tweet_id,text,name


In [41]:
df_archive_clean['name'].value_counts(dropna=False).keys()

Index([       nan,     'Lucy',  'Charlie',   'Cooper',   'Oliver',   'Tucker',
          'Penny',    'Sadie',     'Lola',  'Winston',
       ...
       'Ashleigh',    'Tilly',      'Ace',    'Bubba',    'Dewey',    'Mitch',
           'Hanz',   'Rooney',      'Alf',     'Liam'],
      dtype='object', length=955)

#### Define

- The tweet's source is extracted from the 'source' column by extracting the inner text of the html 'a' tag using regex. 

#### Code

In [42]:
def fix_source(row):
    """
    Returns the extracted string from the html tag
    """
    pattern = re.compile(r'(?:(?!>).)*?(?=(?:<)|$)')
    source = re.findall(pattern, row['source'])[1]
    if source:
        return source
    else:
        return row['source']
    
    
df_archive_clean['source'] = df_archive_clean.apply(lambda x: fix_source(x), axis=1)

#### Test

In [43]:
df_archive_clean['source'].value_counts()

Twitter for iPhone     1962
Vine - Make a Scene      91
Twitter Web Client       30
TweetDeck                11
Name: source, dtype: int64

In [44]:
df_archive_clean.sample(5)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,dog_stage
1680,682003177596559360,2015-12-30 01:00:03+00:00,Twitter for iPhone,Unique dog here. Wrinkly as hell. Weird segmen...,https://twitter.com/dog_rates/status/682003177...,5,10,NaN,NaN
1430,697575480820686848,2016-02-11 00:18:49+00:00,Twitter for iPhone,This is Ole. He's not sure how to gravity. 8/1...,https://twitter.com/dog_rates/status/697575480...,8,10,Ole,NaN
793,773547596996571136,2016-09-07 15:44:53+00:00,Twitter for iPhone,This is Chelsea. She forgot how to dog. 11/10 ...,https://twitter.com/dog_rates/status/773547596...,11,10,Chelsea,pupper
2034,671743150407421952,2015-12-01 17:30:22+00:00,Twitter for iPhone,This is a Tuscaloosa Alcatraz named Jacob (Yac...,https://twitter.com/dog_rates/status/671743150...,11,10,Jacob,NaN
519,810284430598270976,2016-12-18 00:43:57+00:00,Twitter for iPhone,This is Phil. He's a father. A very good fathe...,https://twitter.com/dog_rates/status/810284430...,13,10,Phil,NaN


## 3.2 Tweet Image Predictions

In [45]:
# copy into a new DataFrame to be cleaned
df_pred_clean = df_pred.copy()

#### Define

* Convert the dataframe shape such that each row represents a prediction using the panas `wide_to_long` function.

#### Code

In [46]:
# Renaming the dataset columns
cols = ['tweet_id', 'jpg_url', 'img_num', 
       'prediction_1', 'confidence_1', 'breed_1',
       'prediction_2', 'confidence_2', 'breed_2',
       'prediction_3', 'confidence_3', 'breed_3']

df_pred_clean.columns = cols

df_pred_clean = pd.wide_to_long(df_pred_clean, stubnames=['prediction', 'confidence', 'breed'], 
    i=['tweet_id', 'jpg_url', 'img_num'], j='prediction_level', sep="_").reset_index()

#### Test

In [47]:
df_pred_clean.sample(5)

,tweet_id,jpg_url,img_num,prediction_level,prediction,confidence,breed
4358,776813020089548800,https://pbs.twimg.com/media/CsfLUDbXEAAu0VF.jpg,1,3,standard_poodle,0.168989,True
6184,889638837579907072,https://pbs.twimg.com/media/DFihzFfXsAYGDPR.jpg,1,2,boxer,0.002129,True
3226,717047459982213120,https://pbs.twimg.com/media/CfN23ArXEAEkZkz.jpg,1,2,Labrador_retriever,0.012185,True
4228,770414278348247044,https://pbs.twimg.com/media/CrEPsfWXEAAKvem.jpg,1,2,maillot,0.081449,False
4451,781251288990355457,https://pbs.twimg.com/media/CteP5H5WcAEhdLO.jpg,2,3,seat_belt,0.026730,False


## 3.3 Twitter API

In [48]:
# copy into a new DataFrame to be cleaned
df_api_clean = df_api.copy()

#### Define

* Convert the 'tweet_id', 'retweet_count' and 'favorite_count' columns to integer instead of string.

#### Code

In [49]:
df_api_clean['tweet_id'] = df_api_clean['tweet_id'].astype(int)
df_api_clean['retweet_count'] = df_api_clean['retweet_count'].astype(int)
df_api_clean['favorite_count'] = df_api_clean['favorite_count'].astype(int)

#### Test

In [50]:
df_api_clean.dtypes

tweet_id          int64
retweet_count     int64
favorite_count    int64
dtype: object

## 3.4 Merging Data

#### Define

* Merge the `df_archive_clean` and the `df_api_clean` based on the 'tweet_id' column.

#### Code

In [51]:
df_merged = pd.merge(df_archive_clean, df_api_clean, on='tweet_id')

#### Test

In [52]:
df_merged.sample(3)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,dog_stage,retweet_count,favorite_count
886,674447403907457024,2015-12-09 04:36:06+00:00,Twitter for iPhone,This pupper just wants a belly rub. This puppe...,https://twitter.com/dog_rates/status/674447403...,10,10,NaN,pupper,338,1008
991,672222792075620352,2015-12-03 01:16:17+00:00,Twitter for iPhone,This is Cal. He's a Swedish Geriatric Cheddar....,https://twitter.com/dog_rates/status/672222792...,9,10,Cal,NaN,194,742
730,679503373272485890,2015-12-23 03:26:43+00:00,Twitter for iPhone,This is Dwight. He's a pointy pupper. Very doc...,https://twitter.com/dog_rates/status/679503373...,8,10,Dwight,pupper,1409,3072


## 3.5 Storing Data

export the df_merged file to a new file 'twitter_archive_clean.csv'

In [53]:
df_merged.to_csv('twitter_archive_master.csv',index=False)

export the cleaned image predictions DataFrame to 'image_predictions_clean.csv' 

In [54]:
df_pred_clean.to_csv('image_predictions_clean.csv',index=False)